In [1]:
import numpy as np
import pygame, sys
from pygame.locals import *

pygame.init()

#just define some colors
WHITE = (255,255,255)
RED = (255,0,0)
GREEN = (0, 255, 0)
PURPLE =(102, 0, 204)


### I started to use constants so we could change the size values buttttt
### we would have to adjust screen size and how we get coordinates etc.
### It might be too hard to do all of that

#define board size
BOARDSIZE = 5
#define square size
SQUARESIZE = 48
#define number of clicks
clickCounter = 0
### from https://inventwithpython.com/pygame/chapter2.html
DISPLAYSURF = pygame.display.set_mode((300, 400))
#create display surface
pygame.display.set_caption('Lights Out') #window title
#Set font 
font = pygame.font.Font('freesansbold.ttf', 15) 
text = font.render("Number of clicks: "+str(clickCounter), True,WHITE) 
textRect = text.get_rect()   
# set the center of the rectangular object. 
textRect.center = (150,350)
#DISPLAYSURF.blit(text, textRect) 

import numpy as np

class GameBoard():
    def __init__(self, start, soln, moves):
        self.start = start
        self.soln = soln
        self.moves = moves
        
def readBoards():
    lst = []
    with open('/Users/Admin/Downloads/SJSULightsOut-master/BoardSets.txt') as f:
        for i in range(6):
            print(f.readline())
            
        for i in range(30):
            f.readline()
            moves = f.readline()
            start = readBoardString(f.readline())
            soln = readBoardString(f.readline())
            lst.append(GameBoard(start, soln, moves))
            f.readline()

    return lst

def readBoardString(string):
    lst = []
    for i in range(len(string)):
        if string[i]=='0' or string[i]=='1':
            lst.append(string[i])
    lst=np.asarray(lst)
    return convert2D(lst)

def convert2D(lst):
    arr=np.array(lst.reshape(5,5))
    return arr

import random as random

def getBoard():
    allBoards=list(range(0,30))
    gameBoard=random.sample(allBoards, 1)
    allBoards.remove(gameBoard[0])
    return gameBoard[0]

    
    
class Square():
    def __init__(self, x, y, size, isOn):
        self.size = size
        self.x = x
        self.y = y
        self.isOn = False

    def clicked(self): #just redraws square with new color
        if self.isOn == True:
            pygame.draw.rect(DISPLAYSURF, PURPLE,\
                             (self.x,self.y, \
                              self.size, self.size))
            self.isOn = False
        elif self.isOn == False:
            pygame.draw.rect(DISPLAYSURF, WHITE,\
                             (self.x,self.y, \
                              self.size, self.size))
            self.isOn = True           
    def getx(self):
        return(self.x)
    def gety(self):
        return(self.y)
    def turnOn(self):
        self.isOn = True;
        pygame.draw.rect(DISPLAYSURF, WHITE,\
                             (self.x,self.y, \
                              self.size, self.size))
        return True

#make a whole bunch of separate square objects
def initSquares():
    for i in range(0,BOARDSIZE):
        for j in range(0,BOARDSIZE):
            #this doesn't scale well with BOARDSIZE haha
            #can fix later
            squares[i][j] = Square(25+(i*50),25+(j*50),SQUARESIZE, False)
            #print(squares[i][j].x, squares[i][j].y)
            pygame.draw.rect(DISPLAYSURF, PURPLE, \
                             (squares[i][j].x, squares[i][j].y, \
                              squares[i][j].size, squares[i][j].size))


def printSquares(squares):
    for i in range(BOARDSIZE):
        for j in range(BOARDSIZE):
            print(squares[i][j].isOn, end='')
        print(' ')
            
def level(level):
    print(level.start)
    for i in range(5):
        for j in range(5):
            if level.start[i][j]=='1':
                squares[i][j].turnOn()
    printSquares(squares)
                
#    if(level == 1):
#        global clickCounter
#        clickCounter = 0
#        squares[0][0].turnOn()
#        squares[1][1].turnOn()
#        squares[2][2].turnOn()
#        squares[3][3].turnOn()
#        squares[4][4].turnOn()

        
def click():
    global clickCounter
    clickCounter += 1
    text = font.render("Number of clicks: "+str(clickCounter), True,WHITE) 
    #clears the old text
    DISPLAYSURF.fill(pygame.Color("black"), (0, 300, 300, 100))
    #DISPLAYSURF.fill(pygame.color("black"))
    #display new text
    DISPLAYSURF.blit(text, textRect) 
    
def clicked_square():
    pos = pygame.mouse.get_pos() #where mouse clicked
    x = pos[0]
    y = pos[1] 
    
    if(getSquare(x,y) is None): #None would be outside game board
        return #do nothing
    
    xi,yj = getSquare(x,y)#find out which square was clicked
    squares[xi][yj].clicked() #click the square
        
    """changing the squares around the one that was clicked"""
    if(xi > 0): squares[xi-1][yj].clicked()
    if(yj > 0): squares[xi][yj-1].clicked()
    if(xi < 4): squares[xi+1][yj].clicked()
    if(yj < 4): squares[xi][yj+1].clicked()
    
    click()
    checkWin(); #check if this was the winning move
    
def getSquare(x,y):
    #determine which square you're in by comparing coordinates to 
    for i in range(0, BOARDSIZE):
        for j in range(0, BOARDSIZE):
            if((x > squares[i][j].getx()) & (x < squares[i][j].getx()+45)\
            &(y > squares[i][j].gety()) & (y < squares[i][j].gety()+45)):
                return(i,j)
    return; #not in a square

    
def checkWin():
    #check if game is won
    for i in range(0, BOARDSIZE):
        for j in range(0, BOARDSIZE):
            #print(i, j, squares[i][j].isOn)
            if(squares[i][j].isOn):
                return;
            elif(i==j==4):
                #create win function to do something
                print("win")
                level(1)
    
                
"""   start here   """               
#array to hold squares
squares = np.ndarray(shape=(BOARDSIZE,BOARDSIZE), dtype=Rect, order='F')  
boards=readBoards()
initSquares()   #create game board   
game=getBoard()
print(game)
level(boards[game])

while True: # main game loop
    for event in pygame.event.get():
        if event.type == pygame.MOUSEBUTTONDOWN:
            clicked_square()

        if event.type == QUIT:
            pygame.quit()
            sys.exit()
    pygame.display.update()


### added counter reset
### added levels



        

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html
# Scheme of file: every set of boards organized in 5 rows. (first board starts on row 7)

#   First row: Board number

#   Second row: Target moves (minimum moves to win at start)

#   Third row: Start (initial state of board)

#   Fourth row: Solution (moves to make from start to win)   

#   Fifth row: Blank row (separator for reading clarity)

21
[['0' '0' '1' '0' '0']
 ['0' '1' '1' '1' '0']
 ['1' '1' '1' '0' '1']
 ['1' '0' '0' '0' '1']
 ['0' '0' '1' '0' '1']]
FalseFalseFalseFalseFalse 
FalseFalseFalseFalseFalse 
FalseFalseFalseFalseFalse 
FalseFalseFalseFalseFalse 
FalseFalseFalseFalseFalse 
FalseFalseTrueFalseFalse 
FalseTrueTrueTrueFalse 
TrueTrueTrueFalseTrue 
TrueFalseFalseFalseTrue 
FalseFalseTrueFalseTrue 


SystemExit: 

/Users/Admin/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3334: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [2]:
import numpy as np

class GameBoard():
    def __init__(self, start, soln, moves):
        self.start = start
        self.soln = soln
        self.moves = moves
        
def readBoards():
    lst = []
    print('hi')
    with open('/Users/Admin/Downloads/SJSULightsOut-master/BoardSets.txt') as f:
        for i in range(6):
            print(f.readline())
            
        for i in range(30):
            f.readline()
            moves = f.readline()
            start = readBoardString(f.readline())
            soln = readBoardString(f.readline())
            lst.append(GameBoard(start, soln, moves))
    return lst

def readBoardString(string):
    lst = []
    for i in range(len(string)):
        if string[i]=='0' or string[i]=='1':
            lst.append(string[i])
    print(lst)
#    return lst
    return convert2D(lst)

def convert2D(lst):
    arr=np.array(lst.reshape(5,5))
    return arr

import random as random

def getBoard():
    allBoards=list(range(1,31))
    gameBoard=random.sample(allBoards, 1)
    allBoards.remove(gameBoard[0])
    return gameBoard


        

In [2]:
def readBoards():
    lst = []
    print('hi')
    with open('/Users/Admin/Downloads/SJSULightsOut-master/BoardSets.txt') as f:
        for i in range(6):
            print(f.readline())
            
        for i in range(30):
            f.readline()
            moves = f.readline()
            start = readBoardString(f.readline())
            soln = readBoardString(f.readline())
            lst.append(GameBoard(start, soln, moves))
    return lst



In [ ]:
lst=readBoards()
lst

# 